# Predictive Modeling

This notebook builds and evaluates machine learning models for predicting job displacement risk.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import pickle

sys.path.append('../src')

from data_loader import load_mckinsey_data
from feature_engineering import prepare_ml_features
from models import JobDisplacementPredictor, TimeSeriesForecaster, predict_job_loss_risk
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

sns.set_style("whitegrid")
np.random.seed(42)

## Load and Prepare Data

In [ ]:
# Load data
mckinsey = load_mckinsey_data()

# Prepare features
X, y, feature_cols = prepare_ml_features(mckinsey)

print(f"Feature matrix shape: {X.shape}")
print(f"Target distribution:")
print(y.value_counts().sort_index())

## Train Multiple Models

In [ ]:
# Initialize and train predictor
predictor = JobDisplacementPredictor()
results = predictor.train(X, y, test_size=0.2)

# Display results
print("\nModel Training Results:")
print("=" * 60)
for model_name, metrics in results.items():
    print(f"\n{model_name.upper()}:")
    print(f"  Train Score: {metrics['train_score']:.4f}")
    print(f"  Test Score: {metrics['test_score']:.4f}")
    print(f"  CV Mean: {metrics['cv_mean']:.4f} (+/- {metrics['cv_std']:.4f})")

print(f"\nBest Model: {predictor.best_model_name.upper()}")

## Model Evaluation

In [ ]:
# Prepare test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Evaluate best model
evaluation = predictor.evaluate(X_test, y_test)

# Display classification report
print("\nClassification Report:")
print("=" * 60)
report_df = pd.DataFrame(evaluation['classification_report']).T
display(report_df)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(evaluation['confusion_matrix'], annot=True, fmt='d', cmap='Blues',
            xticklabels=['Low', 'Medium', 'High', 'Very High'],
            yticklabels=['Low', 'Medium', 'High', 'Very High'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## Feature Importance Analysis

In [ ]:
# Get feature importance
importance_df = predictor.get_feature_importance(X.columns)

# Plot feature importance
plt.figure(figsize=(10, 8))
importance_df.head(20).plot(x='feature', y='importance', kind='barh',
                            color='steelblue', ax=plt.gca())
plt.title('Top 20 Feature Importances')
plt.xlabel('Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 10 Most Important Features:")
display(importance_df.head(10))

## Predictions on Sample Occupations

In [ ]:
# Make predictions on all occupations
predictions, probabilities = predictor.predict(X)

# Create results dataframe
results_df = mckinsey[['occupation_name', 'sector', 'automation_potential']].copy()
results_df['predicted_risk'] = predictions
results_df['risk_probability'] = probabilities.max(axis=1)

# Map predictions to labels
risk_labels = {1: 'Low', 2: 'Medium', 3: 'High', 4: 'Very High'}
results_df['risk_label'] = results_df['predicted_risk'].map(risk_labels)

# Display high-risk occupations
print("\nTop 20 Highest Risk Occupations:")
high_risk = results_df.nlargest(20, 'risk_probability')[
    ['occupation_name', 'sector', 'automation_potential', 'risk_label', 'risk_probability']
]
display(high_risk)

# Display low-risk occupations
print("\nTop 20 Lowest Risk Occupations:")
low_risk = results_df.nsmallest(20, 'risk_probability')[
    ['occupation_name', 'sector', 'automation_potential', 'risk_label', 'risk_probability']
]
display(low_risk)

## Time Series Forecasting

In [ ]:
from data_loader import load_global_ai_adoption

# Load global data
global_ai = load_global_ai_adoption()

# Initialize forecaster
forecaster = TimeSeriesForecaster()

# Forecast for top countries
countries = ['United States', 'China', 'Germany', 'United Kingdom', 'Singapore']
forecast_results = {}

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, country in enumerate(countries):
    forecast = forecaster.forecast_by_country(global_ai, country, 'ai_adoption_rate', periods=3)
    forecast_results[country] = forecast
    
    # Plot actual vs forecast
    country_data = global_ai[global_ai['country'] == country].sort_values('year')
    
    axes[i].plot(country_data['year'], country_data['ai_adoption_rate'], 
                'o-', label='Actual', linewidth=2, markersize=8)
    axes[i].plot(forecast['years'], forecast['predictions'], 
                's--', label='Forecast', linewidth=2, markersize=8, color='red')
    axes[i].set_title(f'{country}\nAI Adoption Forecast')
    axes[i].set_xlabel('Year')
    axes[i].set_ylabel('AI Adoption Rate (%)')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

# Remove empty subplot
axes[5].remove()

plt.tight_layout()
plt.show()

# Display forecast results
print("\nAI Adoption Forecasts (2026-2028):")
print("=" * 60)
for country, forecast in forecast_results.items():
    print(f"\n{country}:")
    print(f"  R² Score: {forecast['r2_score']:.4f}")
    for year, pred in zip(forecast['years'], forecast['predictions']):
        print(f"  {year}: {pred:.1f}%")

## Model Performance Comparison

In [ ]:
# Compare model performances
model_comparison = pd.DataFrame({
    'Model': list(results.keys()),
    'Train Score': [m['train_score'] for m in results.values()],
    'Test Score': [m['test_score'] for m in results.values()],
    'CV Mean': [m['cv_mean'] for m in results.values()],
    'CV Std': [m['cv_std'] for m in results.values()]
})

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Train vs Test scores
x = np.arange(len(model_comparison))
width = 0.35

axes[0].bar(x - width/2, model_comparison['Train Score'], width, 
           label='Train Score', color='steelblue')
axes[0].bar(x + width/2, model_comparison['Test Score'], width,
           label='Test Score', color='coral')
axes[0].set_xlabel('Model')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Model Accuracy Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(model_comparison['Model'], rotation=45)
axes[0].legend()

# CV Scores with error bars
axes[1].bar(x, model_comparison['CV Mean'], yerr=model_comparison['CV Std'],
           capsize=5, color='forestgreen', alpha=0.7)
axes[1].set_xlabel('Model')
axes[1].set_ylabel('Cross-Validation Score')
axes[1].set_title('Cross-Validation Scores')
axes[1].set_xticks(x)
axes[1].set_xticklabels(model_comparison['Model'], rotation=45)

plt.tight_layout()
plt.show()

print("\nModel Comparison:")
display(model_comparison)

## Save Model

In [ ]:
# Save the trained model
import os
os.makedirs('../models', exist_ok=True)

with open('../models/job_displacement_predictor.pkl', 'wb') as f:
    pickle.dump({
        'predictor': predictor,
        'feature_names': X.columns.tolist(),
        'results': results
    }, f)

print("Model saved to ../models/job_displacement_predictor.pkl")

# Save predictions
os.makedirs('../data/processed', exist_ok=True)
results_df.to_csv('../data/processed/predictions.csv', index=False)
print("Predictions saved to ../data/processed/predictions.csv")

## Summary

### Model Performance
- Best model: {predictor.best_model_name}
- Cross-validation accuracy: {results[predictor.best_model_name]['cv_mean']:.2%}

### Key Findings
1. Automation potential is the strongest predictor of job displacement risk
2. Sector and education level also significantly impact displacement risk
3. Model successfully identifies high-risk occupations with {results[predictor.best_model_name]['cv_mean']:.1%} accuracy
4. AI adoption is forecasted to continue growing across all major economies